# Gradient-based word deletion

Trained **new** new models that are much better than the previous. They're still training, and hopefully will continue to get better, but I want to test them now to see how they do!

In [1]:
import sys
sys.path.append('/home/jxm3/research/deidentification/unsupervised-deidentification')

In [2]:
from datamodule import WikipediaDataModule
import os

num_cpus = os.cpu_count()

dm = WikipediaDataModule(
    document_model_name_or_path="roberta-base",
    profile_model_name_or_path="google/tapas-base",
    max_seq_length=128,
    dataset_name='wiki_bio',
    dataset_train_split='train[:1024]', # not used in this notebook
    dataset_val_split='val[:20%]',
    dataset_version='1.2.0',
    word_dropout_ratio=0.0,
    word_dropout_perc=0.0,
    num_workers=1,
    train_batch_size=64,
    eval_batch_size=64
)
dm.setup("fit")

Initializing WikipediaDataModule with num_workers = 1 and mask token `<mask>`
loading wiki_bio[1.2.0] split train[:1024]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)


loading wiki_bio[1.2.0] split val[:20%]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-793b771e10f80bbe.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-7d07543b6205ca87.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-7440752484ad8676.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-2c6f94b0d2dcc153.arrow


In [4]:
from model import CoordinateAscentModel

from model_cfg import model_paths_dict

model_key = "model_5"

checkpoint_path = model_paths_dict[model_key]
print(f"loading {model_key} from path:", checkpoint_path)


model = CoordinateAscentModel.load_from_checkpoint(
    checkpoint_path,
    document_model_name_or_path="roberta-base",
    profile_model_name_or_path="google/tapas-base",
    learning_rate=1e-5,
    pretrained_profile_encoder=False,
    lr_scheduler_factor=0.5,
    lr_scheduler_patience=1,
    train_batch_size=1,
    num_workers=1,
    gradient_clip_val=10.0,
)

loading model_5 from path: /home/jxm3/research/deidentification/unsupervised-deidentification/saves/ca__roberta__tapas__adv/deid-wikibio-2_default/236desyb_444/checkpoints/epoch=10-step=50082.ckpt


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialized model with learning_rate = 1e-05 and patience 1


## 2. Define attack in TextAttack 

In [5]:
import textattack

### (a) Beam search + replace with `[MASK]`

In [6]:
class WordSwapSingleWord(textattack.transformations.word_swap.WordSwap):
    """Takes a sentence and transforms it by replacing with a single fixed word.
    """
    single_word: str
    def __init__(self, single_word: str = "?", **kwargs):
        super().__init__(**kwargs)
        self.single_word = single_word

    def _get_replacement_words(self, _word: str):
        return [self.single_word]

transformation = WordSwapSingleWord(single_word=dm.document_tokenizer.mask_token)
transformation(textattack.shared.AttackedText("Hello my name is Jack"))

[<AttackedText "<mask> my name is Jack">,
 <AttackedText "Hello <mask> name is Jack">,
 <AttackedText "Hello my <mask> is Jack">,
 <AttackedText "Hello my name <mask> Jack">,
 <AttackedText "Hello my name is <mask>">]

### (b) "Attack success" as fullfilment of the metric

In [7]:
from typing import List
import torch

class ChangeClassificationToBelowTopKClasses(textattack.goal_functions.ClassificationGoalFunction):
    k: int
    def __init__(self, *args, k: int = 1, **kwargs):
        self.k = k
        super().__init__(*args, **kwargs)

    def _is_goal_complete(self, model_output, _):
        original_class_score = model_output[self.ground_truth_output]
        num_better_classes = (model_output > original_class_score).sum()
        return num_better_classes >= self.k

    def _get_score(self, model_output, _):
        return 1 - model_output[self.ground_truth_output]
    
    
    """have to reimplement the following method to change the precision on the sum-to-one condition."""
    def _process_model_outputs(self, inputs, scores):
        """Processes and validates a list of model outputs.
        This is a task-dependent operation. For example, classification
        outputs need to have a softmax applied.
        """
        # Automatically cast a list or ndarray of predictions to a tensor.
        if isinstance(scores, list):
            scores = torch.tensor(scores)

        # Ensure the returned value is now a tensor.
        if not isinstance(scores, torch.Tensor):
            raise TypeError(
                "Must have list, np.ndarray, or torch.Tensor of "
                f"scores. Got type {type(scores)}"
            )

        # Validation check on model score dimensions
        if scores.ndim == 1:
            # Unsqueeze prediction, if it's been squeezed by the model.
            if len(inputs) == 1:
                scores = scores.unsqueeze(dim=0)
            else:
                raise ValueError(
                    f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
                )
        elif scores.ndim != 2:
            # If model somehow returns too may dimensions, throw an error.
            raise ValueError(
                f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
            )
        elif scores.shape[0] != len(inputs):
            # If model returns an incorrect number of scores, throw an error.
            raise ValueError(
                f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
            )
        elif not ((scores.sum(dim=1) - 1).abs() < 1e-4).all():
            # Values in each row should sum up to 1. The model should return a
            # set of numbers corresponding to probabilities, which should add
            # up to 1. Since they are `torch.float` values, allow a small
            # error in the summation.
            scores = torch.nn.functional.softmax(scores, dim=1)
            if not ((scores.sum(dim=1) - 1).abs() < 1e-4).all():
                raise ValueError("Model scores do not add up to 1.")
        return scores.cpu()


## (c) Model wrapper that computes similarities of input documents with validation profiles

In [8]:
import numpy as np
import tqdm

def precompute_profile_embeddings():
    model.profile_model.cuda()
    model.profile_model.eval()

    model.val_profile_embeddings = np.zeros((len(dm.val_dataset), model.profile_embedding_dim))
    for val_batch in tqdm.tqdm(dm.val_dataloader()[0], desc="Precomputing val embeddings", colour="green", leave=False):
        with torch.no_grad():
            profile_embeddings = model.forward_profile(batch=val_batch)
        model.val_profile_embeddings[val_batch["text_key_id"]] = profile_embeddings.cpu()
    model.val_profile_embeddings = torch.tensor(model.val_profile_embeddings, dtype=torch.float32)
    model.profile_model.train()

precompute_profile_embeddings()

In [9]:
import transformers
from model.model import Model

class MyModelWrapper(textattack.models.wrappers.ModelWrapper):
    model: Model
    tokenizer: transformers.AutoTokenizer
    profile_embeddings: torch.Tensor
    max_seq_length: int
    
    def __init__(self, model: Model, tokenizer: transformers.AutoTokenizer, max_seq_length: int = 128):
        self.model = model
        self.model.eval()
        self.tokenizer = tokenizer
        self.profile_embeddings = torch.tensor(model.val_profile_embeddings)
        self.max_seq_length = max_seq_length
                 
    def to(self, device):
        self.model.to(device)
        self.profile_embeddings.to(device)
        return self # so semantics `model = MyModelWrapper().to('cuda')` works properly

    def __call__(self, text_input_list: List[str], batch_size=32):
        model_device = next(self.model.parameters()).device
        
        doc_tokenized = self.tokenizer.batch_encode_plus(
            text_input_list,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
        doc_tokenized = {f'document__{k}': v for k,v in doc_tokenized.items()}
        with torch.no_grad():
            document_embeddings = self.model.forward_document(batch=doc_tokenized, document_type='document')
            document_to_profile_logits = document_embeddings @ self.profile_embeddings.T.to(model_device)
            document_to_profile_probs = torch.nn.functional.softmax(
                document_to_profile_logits, dim=-1
            )
        assert document_to_profile_probs.shape == (len(text_input_list), len(self.profile_embeddings))
        return document_to_profile_probs
            

## (d) Dataset that loads Wikipedia documents with names as labels

Oh, and it filters out examples that are too long.

In [10]:
from typing import Tuple

from collections import OrderedDict

import datasets

class WikiDataset(textattack.datasets.Dataset):
    dataset: datasets.Dataset
    
    def __init__(self, dm: WikipediaDataModule):
        self.shuffled = True
        self.dataset = [ex for ex in dm.val_dataset]
        self.label_names = list(dm.val_dataset['name'])
    
    def __len__(self) -> int:
        return len(self.dataset)
    
    def __getitem__(self, i: int) -> Tuple[OrderedDict, int]:
        input_dict = OrderedDict([
            ('document', self.dataset[i]['document'])
        ])
        return input_dict, self.dataset[i]['text_key_id']
        

## 3. Run attack once

In [11]:
from textattack.loggers import CSVLogger
from textattack.shared import AttackedText

import pandas as pd
class CustomCSVLogger(CSVLogger):
    """Logs attack results to a CSV."""

    def log_attack_result(self, result: textattack.goal_function_results.ClassificationGoalFunctionResult):
        # TODO print like 'mask1', 'mask2',
        original_text, perturbed_text = result.diff_color(self.color_method)
        original_text = original_text.replace("\n", AttackedText.SPLIT_TOKEN)
        perturbed_text = perturbed_text.replace("\n", AttackedText.SPLIT_TOKEN)
        result_type = result.__class__.__name__.replace("AttackResult", "")
        row = {
            "original_person": result.original_result._processed_output[0],
            "original_text": original_text,
            "perturbed_person": result.perturbed_result._processed_output[0],
            "perturbed_text": perturbed_text,
            "original_score": result.original_result.score,
            "perturbed_score": result.perturbed_result.score,
            "original_output": result.original_result.output,
            "perturbed_output": result.perturbed_result.output,
            "ground_truth_output": result.original_result.ground_truth_output,
            "num_queries": result.num_queries,
            "result_type": result_type,
        }
        self.df = pd.concat([self.df, pd.DataFrame([row])], ignore_index=True)
        self._flushed = False

In [12]:
class MaxNumWordsModified(textattack.constraints.PreTransformationConstraint):
    def __init__(self, max_num_words: int):
        self.max_num_words = max_num_words

    def _get_modifiable_indices(self, current_text):
        """Returns the word indices in current_text which are able to be
        modified."""

        if len(current_text.attack_attrs["modified_indices"]) >= self.max_num_words:
            return set()
        else:
            return set(range(len(current_text.words)))

    def extra_repr_keys(self):
        return ["max_num_words"]

In [13]:
model_wrapper = MyModelWrapper(model=model, tokenizer=dm.document_tokenizer)
model_wrapper.to('cuda')

<ipython-input-9-1c87bb4263f0>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.profile_embeddings = torch.tensor(model.val_profile_embeddings)


In [15]:
# 
#  Initialize attack
# 

from textattack import Attack
from textattack.constraints.pre_transformation import RepeatModification

goal_function = ChangeClassificationToBelowTopKClasses(model_wrapper, k=1)
constraints = [RepeatModification(), MaxNumWordsModified(max_num_words=50)]
transformation = WordSwapSingleWord(single_word=dm.document_tokenizer.mask_token)
search_method = textattack.search_methods.BeamSearch(beam_width=4)

attack = Attack(
    goal_function, constraints, transformation, search_method
)

from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples=15, disable_stdout=True)
dataset = WikiDataset(dm)

attacker = Attacker(attack, dataset, attack_args)

results_iterable = attacker.attack_dataset()

logger = CustomCSVLogger(color_method='html')

# 
# Run attack
# 
from tqdm import tqdm
for result in results_iterable:
    tqdm._instances.clear() # Doesn't fix the progress bar :-(
    logger.log_attack_result(result)

from IPython.display import display, HTML

display(HTML(logger.df.to_html(escape=False)))

textattack: No entry found for goal function <class '__main__.ChangeClassificationToBelowTopKClasses'>.
textattack: Unknown if model of class <class 'model.coordinate_ascent.CoordinateAscentModel'> compatible with goal function <class '__main__.ChangeClassificationToBelowTopKClasses'>.


Attack(
  (search_method): BeamSearch(
    (beam_width):  4
  )
  (goal_function):  ChangeClassificationToBelowTopKClasses
  (transformation):  WordSwapSingleWord
  (constraints): 
    (0): RepeatModification
    (1): MaxNumWordsModified(
        (max_num_words):  50
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 15 / 0 / 0 / 15: 100%|██████████| 15/15 [01:19<00:00,  5.31s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 17.41% |
| Average num. words per input: | 51.93  |
| Avg num queries:              | 1366.8 |
+-------------------------------+--------+


textattack: Logging to CSV at path results.csv


,original_person,original_text,perturbed_person,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,Michael iii of alexandria,"pope michael iii of alexandria (also known as khail iii) was the coptic pope of alexandria and patriarch of the see of st. mark (880 -- 907) .in 882 , the governor of egypt , ahmad ibn tulun , forced khail to pay heavy contributions , forcing him to sell a church and some attached properties to the local jewish community .this building was at one time believed to have later become the site of the cairo geniza .",Saint columba of spain,"pope <mask> <mask> of <mask> (also known as <mask> <mask>) was the <mask> pope of <mask> <mask> <mask> of the <mask> <mask> <mask>. <mask> (880 -- <mask>) .in 882 , the governor of <mask> , ahmad ibn tulun , forced khail to pay heavy contributions , forcing him to sell a church and some attached properties to the local jewish community .this building was at one time believed to have later become the site of the cairo geniza .",1.473963e-03,0.932209,0,14536,0,3742,Successful
1,Hui jun,hui jun is a male former table tennis player from china .,Lany kaligis,<mask> <mask> is a <mask> former <mask> tennis player from <mask> .,1.641512e-04,0.988513,1,6736,1,148,Successful
2,Okan öztürk,okan Öztürk (born 30 november 1977) is a turkish professional footballer .he currently plays as a striker for yeni malatyaspor .,Adem büyük,<mask> <mask> (born 30 november <mask>) is a turkish professional footballer .he currently plays as a striker for yeni malatyaspor .,1.430511e-06,0.863823,2,2279,2,169,Successful
3,Marie stephan,"marie stephan , (born march 14 , 1996) is a professional squash player who represents france .she reached a career-high world ranking of world no. 101 in july 2015 .",Laura pomportes,"<mask> <mask> , (born <mask> 14 , <mask>) is a professional squash player who represents france .she reached a career-high world ranking of world no. 101 in july <mask> .",5.960464e-07,0.777259,3,4726,3,420,Successful
4,Leonard l. martino,leonard l. martino is a former democratic member of the pennsylvania house of representatives .he was born in butler to michael and angela pitullio martino .,J. scot chadwick,<mask> <mask>. <mask> is a former <mask> member of the pennsylvania house of representatives .he was born in butler to michael and angela pitullio martino .,6.556511e-06,0.938695,4,12490,4,302,Successful
5,Salome jens,"salome jens (born may 8 , 1935) is an american stage , film and television actress .she is perhaps best known for portraying the female changeling on '' '' .",Jenny wright,"<mask> <mask> (born may 8 , <mask>) is an american stage , film and television actress .she is perhaps best known for portraying the female changeling on '' '' .",0.000000e+00,0.934823,5,13625,5,214,Successful
6,Carl crawford,"carl demonte crawford (born august 5 , 1981) , nicknamed `` the perfect storm '' , is an american professional baseball left fielder with the los angeles dodgers of major league baseball (mlb) .he bats and throws left-handed .crawford was drafted by the tampa bay devil rays in the second round (52nd overall) of the 1999 major league baseball draft .he made his major league debut in 2002 .crawford has more triples (121) than any other active baseball player .",Steven gerrard mbe,"<mask> demonte <mask> (born <mask> 5 , <mask>) , <mask> `` the perfect storm '' , is an american professional baseball <mask> <mask> with the los angeles <mask> of major league baseball (mlb) .he bats and throws left-handed .<mask> was drafted by the <mask> <mask> devil rays in the second round (52nd overall) of the 1999 major league baseball draft .he made his major league debut in 2002 .<mask> has more <mask> (121) than any other active baseball player .",0.000000e+00,0.663866,6,10955,6,3364,Successful
7,Jim bob,"jim bob (born james neil morrison on 22 november 1960) is a british musician and author , best known as the singer

In [ ]:
display(HTML(logger.df.to_html(escape=False)))